*Credit*: some material here has been adapted from the Appendix of [Rasmussen and Williams "Gaussian Processes for Machine Learning"](http://www.gaussianprocess.org/gpml/).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Cholesky decomposition

The Cholesky decomposition of a symmetric, positive definite matrix $\mathbf{A}$ decomposes $\mathbf{A}$ into a product of a lower triangular matrix $\mathbf{L}$ and its transpose:

$$\mathbf{L}\mathbf{L}^{\top} = \mathbf{A}$$

where $\mathbf{L}$ is called the Cholesky factor. The Cholesky decomposition is also known as the "matrix square root".

In Python, the Cholesky decomposition can be efficiently computed via `scipy.linalg.cho_factor`.

In [ ]:
from scipy.linalg import cho_factor
A = np.array([[9, 3, 1, 5], [3, 7, 5, 1], [1, 5, 9, 2], [5, 1, 2, 6]])
print(A)
c, low = cho_factor(A)

The first argument returned by `cho_factor` is a matrix whose upper or lower triangle contains the Cholesky factor. The second argument returned is a boolean flag indicating whether the factor is in the lower or upper triangle.

As a warning, the function also returns random data in the entries not used by the Cholesky decomposition. But if you need to zero these entries, use `scipy.linalg.cholesky` instead.


In [ ]:
print(c)
print(low)

In this case, the Cholesky factor is in the upper triangle.

In [ ]:
np.allclose(np.triu(c).T @ np. triu(c) - A, np.zeros((4, 4)))

Note that here we used the short form for the matrix product, `@`.

## Solving linear systems with a symmetric, positive definite coffeicient matrix

The Cholesky decomposition is useful for solving linear systems with symmetric, [positive definite](http://en.wikipedia.org/wiki/Positive-definite_matrix) coefficient matrix $\mathbf{A}$. To solve $\mathbf{A}\mathbf{x}=\mathbf{b}$ for $\mathbf{x}$, first solve the triangular system $\mathbf{L}^{\top}\mathbf{y}=\mathbf{b}$ by forward substitution and then the triangular system $\mathbf{L}^{\top}\mathbf{x} = \mathbf{y}$ by back substitution.

Let's start by creating a symmetric, positive definite matrix.

In [ ]:
B = np.random.randn(4,3)
A = np.dot(B.T, B)  # This will be symmetric, positive definite
print(A)
b = np.random.randn(3)
print(b)

This is how we would usually solve the linear system $\mathbf{A}\mathbf{x}=\mathbf{b}$, but it does not exploit the symmetric, positive definite structure of $\mathbf{A}$.

In [ ]:
x = np.linalg.solve(A, b)

SciPy provides the `cho_solve` method to automate the above procedure. It accepts as an input argument a Cholesky decomposition as output by `cho_factor`.

In [ ]:
from scipy.linalg import cho_solve
x_ = cho_solve(cho_factor(A, lower=True), b)
print(x)
print(x_)
np.allclose(x, x_)

## Generating Gaussian samples

To generate samples $\mathbf{x} \sim \mathcal{N} \left(\boldsymbol{\mu}, \boldsymbol{\Sigma} \right)$ with arbitrary mean $\boldsymbol{\mu}$ and covariance matrix $\boldsymbol{\Sigma}$ using a scalar Gaussian generator (which is readily available in many programming environments), we can first compute the Cholesky decomposition $L$ of the symmetric, positive definite covariance matrix $\boldsymbol{\Sigma}=\mathbf{L}\mathbf{L}^{\top}$. Then we generate $\mathbf{u} \sim \mathcal{N}\left(\mathbf{0}, I\right)$ by multiple separate calls to the scalar Gaussian generator. Compute $\mathbf{x} = \mathbf{\boldsymbol{\mu}} + L\mathbf{u}$, which has the desired distribution with mean $\boldsymbol{\mu}$ and covariance $L \mathbb{E} \left[ \mathbf{u} \mathbf{u}^t \right] L^T = \boldsymbol{\Sigma}$ (by the independence of the elements of $\mathbf{u}$).

In practice it may be necessary to add a small multiple of the identity matrix $\epsilon I$ to the covariance matrix for numerical reasons. This is because the eigenvalues of the matrix $\boldsymbol{\Sigma}$ can decay very rapidly and without this stabilization the Cholesky decomposition fails. The effect on the generated samples is to add additional independent noise of variance $\epsilon$. From the context $\epsilon$ can usually be chosen to have inconsequential effects on the samples, while ensuring numerical stability.

Note that Numpy already provides ``random.multivariate_normal`` and, in general, you should trust the Numpy provided methods as they are efficient and implement various types of error-checking. The following shows the use of Cholesky decomposition for informational purposes.

In [ ]:
mu = np.array([2, 2])
sig = np.array([[0.1, 0.25], 
              [0.25, 0.8]])
x = np.random.multivariate_normal(mu, sig, (100, ))
plt.scatter(x[:, 0], x[:, 1], alpha=0.5)

# show the use of Cholesky decomposition to sample
L = np.linalg.cholesky(sig)
u = np.random.randn(100, 2)

x_ = mu + np.dot(u, L.T)  # Python likes data in rows, dimensions in columns
plt.scatter(x_[:, 0], x_[:, 1], edgecolor='black', color='pink', alpha=0.5)